In [1]:
from classes import *
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon as MplPolygon
import geopandas as gpd
from shapely.geometry import Polygon as ShapelyPolygon
import json
import folium

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
data1_path = "/data/Polygons_no_mountains.geojson"
data2_path = "/data/Polygons_with_mountains.geojson"
data3_path = "swissnames_points_json_export.geojson"

json_files = [(data1_path, "Gemeinden_ohne_Berge"),
(data2_path, "Gemeinden_mit_Berge"),
(data3_path, "Swissnames_Points")]
